In [1]:
import re
import json
import os
import shutil

from datetime import datetime
import IPython.display as ipd

import numpy as np
import pandas as pd
pd.options.display.max_columns = 100

import datasets as hfd

from transformers import (
    Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments, Trainer
)

import torch
print(torch.cuda.is_available())

True


In [2]:
from src.data_collator import DataCollatorCTCWithPadding

In [3]:
import logging
logging_format_str = '%(asctime)s:%(name)s:%(levelname)s:%(message)s'
logging.basicConfig(format=logging_format_str, level=logging.WARNING)
logger = logging.getLogger('STT')
logger.setLevel(level=logging.DEBUG)

In [4]:
DATA_ROOT_DP = os.environ['DATA_HOME']
CV_PROCESSED_2_DP = f'{DATA_ROOT_DP}/datasets/cv-corpus-8.0-2022-01-19__be__processed__2'

In [5]:
ds = hfd.load_from_disk(CV_PROCESSED_2_DP)
ds

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 314305
    })
    dev: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 15803
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 15801
    })
})

In [6]:
# # select subset of data
# ds['train'] = ds['train'].select(range(300))
# ds['dev'] = ds['dev'].select(range(100))
# ds

In [7]:
wer_metric = hfd.load_metric("wer")

In [8]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [9]:
processor = Wav2Vec2Processor.from_pretrained('artifacts/processor')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
vocab = processor.tokenizer.get_vocab()
pd.Series(vocab).to_frame().T

,',i,а,б,в,г,д,е,ж,з,й,к,л,м,н,о,п,р,с,т,у,ф,х,ц,ч,ш,ы,ь,э,ю,я,ё,і,ў,|,[UNK],[PAD],<s>,</s>
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,0,35,36,37,38


In [11]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [12]:
vocab_size = len(processor.tokenizer.get_vocab())
logger.info(f'vocab_size: {vocab_size}')

2022-03-29 04:05:22,809:STT:INFO:vocab_size: 39


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=vocab_size
)

In [14]:
model.freeze_feature_extractor()

In [15]:
now_str = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

train_out_dp = os.path.join('train', now_str)
os.makedirs(train_out_dp, exist_ok=True)
print(f'will save train output to: "{train_out_dp}"')

logs_dp = os.path.join(train_out_dp, 'logs')
print(f'will save train logs to: "{logs_dp}"')

will save train output to: "train/2022-03-29-04-05-24"
will save train logs to: "train/2022-03-29-04-05-24/logs"


In [16]:
steps = 1080

training_args = TrainingArguments(
    output_dir=train_out_dp,
    logging_dir=logs_dp,

    num_train_epochs=1,
    evaluation_strategy='steps',
    save_strategy='steps',
    logging_strategy='steps',
    eval_steps=steps,
    logging_steps=steps,
    save_steps=steps,
    save_total_limit=None,
    load_best_model_at_end=True,
    metric_for_best_model='wer',
    greater_is_better=False,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,
    group_by_length=False,
    gradient_checkpointing=True, 
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=700,  # ????
)

In [17]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ds['train'],
    eval_dataset=ds['dev'],
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [ ]:
trainer.train()